In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dataframe_image as dfi
from datetime import datetime
import scipy
import itertools

In [3]:
file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\all codes\temp_pre_01_22.csv"
temp_pre = pd.read_csv(file)

In [4]:
reg=[]
for admin in temp_pre['admin1'].unique():
    a = temp_pre[temp_pre['admin1']==admin].reset_index(drop=True)
    reg.append(a)

In [5]:
avg_t = np.zeros(18)
avg_p = np.zeros(18)
std_t = np.zeros(18)
std_p = np.zeros(18)

for i in range(18):

    reg[i]['year'] = reg[i]['time'].str[:4]
    reg[i]['month'] = reg[i]['time'].str[5:7]

    mean_temp_i = reg[i].groupby('month')['tmx'].mean()
    std_temp_i = reg[i].groupby('month')['tmx'].std()
    reg[i]['avg_temp'] = reg[i]['month'].map(mean_temp_i)
    reg[i]['std_temp'] = reg[i]['month'].map(std_temp_i)
    
    reg[i]['difference']= (reg[i]['tmx']-reg[i]['avg_temp'])/reg[i]['std_temp']
    reg[i]['TA'] = (reg[i]['difference'].shift(2) + reg[i]['difference'].shift(1) + reg[i]['difference'])/3
    
    mean_pre_i = reg[i].groupby('month')['pre'].mean()
    std_pre_i = reg[i].groupby('month')['pre'].std()
    reg[i]['avg_pre'] = reg[i]['month'].map(mean_pre_i)
    reg[i]['std_pre'] = reg[i]['month'].map(std_pre_i)
    
    reg[i]['diff_p']= (reg[i]['pre']-reg[i]['avg_pre'])/reg[i]['std_pre']
    reg[i]['PA'] = (reg[i]['diff_p'].shift(2) + reg[i]['diff_p'].shift(1) + reg[i]['diff_p'])/3
    
    #reg[i] = reg[i].reindex(columns=['time','admin1','TA','PA'])
    reg[i] = reg[i].reset_index()

In [6]:
df_c_97_22=pd.concat([reg[i] for i in range(18)], axis=0)
df_c_97_22 = df_c_97_22.dropna()

In [7]:
file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\all codes\df_97_22.csv"
df_97_22 = pd.read_csv(file)

In [8]:
df_c_97_22 = pd.merge(df_97_22, df_c_97_22, on=['time','admin1'], how='outer')
df_c_97_22 = df_c_97_22.dropna()
df_c_97_22 = df_c_97_22[['time','admin1','TA','PA','conflicts']]

In [9]:
df_c_97_22['DL'] = 0
mask = df_c_97_22['TA'] > 0
# Create a group identifier for each consecutive group
group_id = (mask != mask.shift()).cumsum()
# Calculate the count within each group
count = df_c_97_22.groupby(group_id).cumcount() + 1
# Assign the count values to the 'DL' column
df_c_97_22['DL'] = np.where(mask, count, 0)
df_c_97_22 = df_c_97_22[['time','admin1','TA','PA','DL','conflicts']]

In [10]:
df_c_97_22['admin1'] = df_c_97_22['admin1'].str.replace('Lower Shabelle', 'Lower_Shabelle')
df_c_97_22['admin1'] = df_c_97_22['admin1'].str.replace('Middle Shabelle', 'Middle_Shabelle')
df_c_97_22['admin1'] = df_c_97_22['admin1'].str.replace('Lower Juba', 'Lower_Juba')
df_c_97_22['admin1'] = df_c_97_22['admin1'].str.replace('Middle Juba', 'Middle_Juba')
df_c_97_22['admin1'] = df_c_97_22['admin1'].str.replace('Woqooyi Galbeed', 'Woqooyi_Galbeed')

In [88]:
#df_c_97_22.to_csv("TA_PA_DL_somalia.csv", index=False)